### this is a youtube tutorial for making a facial recognition model 
##### link: https://www.youtube.com/watch?v=UMjW4Db4E_g&list=PLgNJO2hghbmhHuhURAGbe6KWpiYZt0AMH&index=1

# 1. Setup

## 1.1 Install Dependencies

In [1]:
!pip install tensorflow opencv-python matplotlib

## 1.2 Import Dependencies

In [2]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow. keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## 1.3 Set GPU Growth

In [4]:
# To avoid  OOM errors by setting GPU Memory Consumption Groth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## 1.4 Create Folder Structure

In [5]:
# paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# make dirs
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# 2. Collect Positive and Anchors

## 2.1 Untar Labelled Faces in the Wild Dataset

In [ ]:
# for me the umass website didnt work hence used kaggle dataset
# link: https://www.kaggle.com/datasets/ashwingupta3012/human-faces?resource=download-directory

In [6]:
!pip install pillow

In [7]:
# ressing image to 250px x 250px
from PIL import Image
def resizing(folder_path, size=(250,250)):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    for file_name in files:
        # Construct full file path
        file_path = os.path.join(folder_path, file_name)
        try:
            # Open an image file
            with Image.open(file_path) as img:
                # Resize the image
                img = img.resize(size, Image.Resampling.LANCZOS)
                # Save the resized image back to the same path
                img.save(file_path)
        except IOError:
            print(f"Cannot resize {file_name}, it might not be an image file.")
folder_path= NEG_PATH
resizing(folder_path)

## 2.2 Collect Positive and Anchor Classes

In [8]:
import uuid

In [ ]:
#establishing connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    #cutting down the frame
    frame = frame[120:120+250,200:200+250,:]
    #collecting anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        #ceating a unique file path
        image_name= os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        #write out that is saving the image
        cv2.imwrite(image_name,frame)
    #collecting positive
    if cv2.waitKey(1) & 0XFF == ord('p'):
        #ceating a unique file path
        image_name= os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        #write out that is saving the image
        cv2.imwrite(image_name,frame)

    #show image on screen
    cv2.imshow('Image Collection', frame)
    
    #breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
#releasing the webcam
cap.release()
#closing the image frame
cv2.destroyAllWindows()

# 3. Load and Preprocess Images

## 3.1 Get Image Directories

In [9]:
anchor= tf.data.Dataset.list_files(ANC_PATH+'\\*.jpg').take(300)
positive= tf.data.Dataset.list_files(POS_PATH+'\\*.jpg').take(300)
negative= tf.data.Dataset.list_files(NEG_PATH+'\\*.jpg').take(300)

In [10]:
dir_test = anchor.as_numpy_iterator()

In [11]:
dir_test.next()

b'data\\anchor\\ebec0af7-59f9-11ef-b409-94e70b067e20.jpg'

## 3.2 Preprocess- Scale and Resize

In [66]:
from PIL import Image
import os

# Function to convert an image to RGB or RGBA if it has transparency
def convert_to_rgb(image_path):
    img = Image.open(image_path)
    
    # Check if the image has transparency (e.g., if it has a palette or alpha channel)
    if img.mode in ('P', 'RGBA'):
        img = img.convert('RGBA')  # Convert to RGBA if the image has transparency
    img = img.convert('RGB')   # Otherwise, convert to RGB
    
    return img

# Example usage
image_folder = NEG_PATH
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    rgb_image = convert_to_rgb(image_path)
    rgb_image.save(image_path)  # Save the converted image, replacing the original


In [67]:
def preprocess(file_path):
    
    #reading img from file path
    byte_img = tf.io.read_file(file_path)
    
    #load in the image for processing
    img = tf.io.decode_jpeg(byte_img)
    
    #resizing
    img = tf.image.resize(img,(100,100))
    
    #converting pixel value from 0-255 to 0-1
    img = img/255.0
    return img

## 3.3 Create Labelled Dataset

In [68]:
# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data= positives.concatenate(negatives)

In [69]:
samples = data.as_numpy_iterator()
example = samples.next()
example

(b'data\\anchor\\e98c38a1-59f9-11ef-90a0-94e70b067e20.jpg',
 b'data\\positive\\42efd628-59fa-11ef-a7a0-94e70b067e20.jpg',
 1.0)

## 3.4 Build Train and Test Partition

In [70]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img),preprocess(validation_img), label)

In [71]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [72]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [73]:
train_samples= train_data.as_numpy_iterator()

In [74]:
train_sample=train_samples.next()

In [75]:
len(train_sample[0])

16

In [76]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = data.take(round(len(data)*.3))
test_data = train_data.batch(16)
test_data = train_data.prefetch(8)

 # 4. Model Engineering

## 4.1 Build Embedding Layer

In [77]:
inp = Input(shape=(100,100,3), name="input_img")

In [78]:
c1 = Conv2D(64,(10,10), activation='relu')(inp)

In [79]:
m1 = MaxPooling2D(64,(2,2),padding='same')(c1)

In [80]:
c2 = Conv2D(128,(7,7), activation='relu')(m1)
m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
c3 = Conv2D(128,(4,4), activation='relu')(m2)
m3 = MaxPooling2D(64,(2,2),padding='same')(c3)

In [81]:
#final embedding block
c4 = Conv2D(256,(4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)

In [82]:
mod = Model(inputs=[inp], outputs=[d1], name='embedding')

In [83]:
mod.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_img (InputLayer)               │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 91, 91, 64)          │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 46, 46, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 40, 40, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 17, 17, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 9, 9, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 6, 6, 256)           │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4096)                │      37,752,832 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

In [101]:
def make_embedding():
    inp = Input(shape=(100,100,3), name='input_image')
    # block 1
    c1 = Conv2D(64,(10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64,(2,2),padding='same')(c1)

    # block 2
    c2 = Conv2D(128,(7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2),padding='same')(c2)

    # block 3
    c3 = Conv2D(128,(4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2),padding='same')(c3)

    #final embedding block
    c4 = Conv2D(256,(4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [102]:
embedding = make_embedding()

In [103]:
embedding.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)             │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 91, 91, 64)          │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 46, 46, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 40, 40, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 17, 17, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 9, 9, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 6, 6, 256)           │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4096)                │      37,752,832 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

## 4.2 Build Distance Layer

In [104]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
         
    def call(self, input_embedding, validation_embedding):
        input_embedding = tf.convert_to_tensor(input_embedding)
        validation_embedding = tf.convert_to_tensor(validation_embedding)
        return tf.math.abs(input_embedding - validation_embedding)

In [105]:
l1= L1Dist()

## 4.3 Make Siamese Model

In [106]:
def make_siamese_model():
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    # Validation image in the network
    validation_image = Input(name= 'validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    # Classification Layer
    classifier = Dense (1, activation = 'sigmoid')(distances)
    return Model(inputs = [input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [107]:
siamese_model = make_siamese_model()

In [108]:
siamese_model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_img (InputLayer)        │ (None, 100, 100, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ validation_img (InputLayer)   │ (None, 100, 100, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Functional)        │ (None, 4096)              │      38,960,448 │ input_img[0][0],           │
│                               │                           │                 │ validation_img[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ l1_dist_5 (L1Dist)            │ (1, None, 4096)           │               0 │ embedding[0][0],           │
│                               │                           │                 │ embedding[1][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (1, None, 1)              │           4,097 │ l1_dist_5[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

# 5. Training

## 5.1 Setup Loss and Optimizer

In [109]:
binary_cross_loss = tf.losses.BinaryCrossentropy(from_logits=True)
# optimizers
opt = tf.keras.optimizers.Adam(1e-4) #0.0001


## 5.2 Establish Checkpoints

In [110]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'chkpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

## 5.3 Build Train Step Function

In [111]:
test_batch = train_data.as_numpy_iterator()

In [112]:
batch_1 = test_batch.next()

In [113]:
X = batch_1[:2]
y = batch_1[2]

In [114]:
len(X)

2

In [115]:
siamese_model??

Signature:   siamese_model(*args, **kwargs)
Type:        Functional
String form: <Functional name=SiameseNetwork, built=True>
File:        c:\users\burhanuddin chital\appdata\local\programs\python\python312\lib\site-packages\keras\src\models\functional.py
Source:     
class Functional(Function, Model):
    """A `Functional` model is a `Model` defined as a directed graph of layers.

    Three types of `Model` exist: subclassed `Model`, `Functional` model,
    and `Sequential` (a special case of `Functional`).

    A `Functional` model can be instantiated by passing two arguments to
    `__init__()`. The first argument is the `keras.Input` objects
    that represent the inputs to the model.
    The second argument specifies the output tensors that represent
    the outputs of this model. Both arguments can be a nested structure
    of tensors.

    Example:

    ```
    inputs = {'x1': keras.Input(shape=(10,), name='x1'),
              'x2': keras.Input(shape=(1,), name='x2')}
    t = ke

In [116]:
yhat = siamese_model(X, training=True)
yhat

<tf.Tensor: shape=(1, 16, 1), dtype=float32, numpy=
array([[[0.5008192 ],
        [0.5021616 ],
        [0.5036222 ],
        [0.50218904],
        [0.5031103 ],
        [0.50157094],
        [0.50433636],
        [0.5045364 ],
        [0.50231314],
        [0.5009773 ],
        [0.50105417],
        [0.5017164 ],
        [0.5014215 ],
        [0.50562394],
        [0.50174063],
        [0.50099504]]], dtype=float32)>

In [117]:
# Reshape y and yhat to ensure they have the same shape
yhat = tf.squeeze(yhat)


In [118]:
yhat

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.5008192 , 0.5021616 , 0.5036222 , 0.50218904, 0.5031103 ,
       0.50157094, 0.50433636, 0.5045364 , 0.50231314, 0.5009773 ,
       0.50105417, 0.5017164 , 0.5014215 , 0.50562394, 0.50174063,
       0.50099504], dtype=float32)>

In [119]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        # Get anchor and validation img
        X = batch[:2]
        # Get label 
        y = batch[2]

        # prediction time
        yhat = siamese_model(X, training=True)
        # Reshape yhat to ensure they have the same shape
        yhat = tf.squeeze(yhat)
        # Calculate Loss 
        loss = binary_cross_loss(y, yhat) #y stands for actual results and yhat are the predictions
    print(loss)
    # calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)

    #calc updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    

## 5.4 Buildng Train Loop

In [120]:
def train(data, EPOCHS):
    #Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch{}/{}'.format(epoch,EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        #Loop through each batch
        for idx, batch in enumerate(data):
            # run train step
            train_step(batch)
            progbar.update(idx+1)

        #save chkpt
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

## 5.5 Train the Model

In [121]:
EPOCHS=50

In [122]:
train(train_data,EPOCHS)


 Epoch1/50
Tensor("binary_crossentropy/Mean:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/Mean:0", shape=(), dtype=float32)
26/27 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/stepTensor("binary_crossentropy/Mean:0", shape=(), dtype=float32)
27/27 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step

 Epoch2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step

 Epoch3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step

 Epoch4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step

 Epoch5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step

 Epoch6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step

 Epoch7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step

 Epoch8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step

 Epoch9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step

 Epoch10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step

 Epoch11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step

 Epoch12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step

 Epoch13/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step

 Epoch14/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step

 Epoch15/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step

 Epoc

# 6. Evaluate Model

In [125]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [141]:
# get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [142]:
y_true

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
      dtype=float32)

In [143]:
# make predictions 
y_hat = siamese_model.predict([test_input,test_val])
y_hat=y_hat[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step


In [144]:
# Post processing results 
res = [1 if prediction > 0.5 else 0 for prediction in y_hat]
res

[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0]

In [145]:
y_true

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
      dtype=float32)

In [146]:
# metric object
m= Recall()

# calculating recall val
m.update_state(y_true,y_hat)

# return result
m.result().numpy()

1.0

## 6.4 Viz Results

# 7. Save Model

In [152]:
siamese_model.save('siamesemodel.keras')

In [151]:
siamese_model = tf.keras.models.load_model('siamesemodel.keras', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})